In [ ]:
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

import pandas as pd
import pandas_datareader.data as web
from datetime import datetime

In [ ]:
# Read the company list that is shown in the dropdown menu (This list is static, not fetched from web)
comp_list = pd.read_csv("company_list.csv")

In [ ]:
# Generate the list of dictionaries for dropdown
options = []
for i in comp_list.index:
    # {'label':'user sees', 'value':'script sees'}
    cdict = {'label':comp_list.iloc[i]['NAME'], 'value':comp_list.iloc[i]['SYMBOL']}
    options.append(cdict)

In [ ]:
# launch the application
app = dash.Dash()

In [ ]:
app.layout = html.Div([
    # Header of the dashboard
    html.H1(children='Stock Ticker Dashboard'),
    
    # Element on the left (Dropdown and its header)
    html.Div([
        html.H3(children='Select Stock Symbols'),
        dcc.Dropdown(
            id='dropdown',
            options=options,
            value=['TSLA.US'], # Default company that will shown to user at initialization
            multi=True
        )
    ], style={'width':'30%', 'display':'inline-block', 'verticalAlign':'top', 'paddingRight':'25px'}),
    
    # Element next to dropdown - The date selector and it's header
    html.Div([
        html.H3(children='Select Start and End Dates'),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed = datetime(2018, 1, 1),
            max_date_allowed = datetime.today(),
            start_date = datetime(2020, 1, 1), # Default start date
            end_date = datetime.today() # Default end date
        )
    ], style={'width':'30%', 'display':'inline-block', 'verticalAlign':'top'}),
    
    # Submit button next to the above two elements
    html.Div([
        html.Button(
            id='submit-button', 
            children='SUBMIT',
            n_clicks=0,
            style={'fontSize':24}
        )
    ], style={'display':'inline-block', 'verticalAlign':'bottom', 'marginBottom':'25px'}),
    
    # The graph (It is empty by default, parameters are filled at callback below when submit button is clicked)
    html.Div([
        dcc.Graph(id='graph')
    ])
])

In [ ]:
@app.callback(
    # The figure component of graph is the output that we want to change when submit is clicked
    Output('graph','figure'),
    # Our input is the button click
    [Input('submit-button', 'n_clicks')],
    # These states are the intermediary values to keep in memory before click
    [State('dropdown','value'), State('date-picker','start_date'), State('date-picker','end_date')]
)
def update_graph(nclicks, stock_ticker, start_date, end_date):
    start = datetime.strptime(start_date[:10], '%Y-%m-%d')
    end = datetime.strptime(end_date[:10], '%Y-%m-%d')
    # Traces are the figure data for each dropdown element selected 
    traces = []
    for tic in stock_ticker:
        # Each company's stock price values are fetched from internet using API's of pandas-datareader 
        df = web.DataReader(tic, 'stooq', start, end)
        traces.append({'x':df.index, 'y':df['Close'], 'name':tic})
    # Final figure after selection from dropdown and date picker are made
    fig={
        'data':traces,
        'layout':{'title':', '.join(stock_ticker)+' Closing Prices'}
    }
    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server()